<a href="https://colab.research.google.com/github/objectis/ai-deep-q-learning/blob/master/CartPole.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

###Imports

In [0]:
import random
import gym
import numpy as np
from collections import deque
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import Adam
from keras import backend as K

import tensorflow as tf

###Agent

In [0]:
EPISODES = 60

class DQNAgent:
    def __init__(self, state_size, action_size):
        self.state_size = state_size
        self.action_size = action_size
        self.memory = deque(maxlen=2000)
        self.gamma = 0.95    # discount rate
        self.epsilon = 1.0  # exploration rate
        self.epsilon_min = 0.01
        self.epsilon_decay = 0.99
        self.learning_rate = 0.001
        self.model = self._build_model()
        self.target_model = self._build_model()
        self.update_target_model()

    def _huber_loss(self, y_true, y_pred, clip_delta=1.0):
        error = y_true - y_pred
        cond  = K.abs(error) <= clip_delta

        squared_loss = 0.5 * K.square(error)
        quadratic_loss = 0.5 * K.square(clip_delta) + clip_delta * (K.abs(error) - clip_delta)

        return K.mean(tf.where(cond, squared_loss, quadratic_loss))

    def _build_model(self):
        model = Sequential()
        model.add(Dense(24, input_dim=self.state_size, activation='relu'))
        model.add(Dense(24, activation='relu'))
        model.add(Dense(self.action_size, activation='linear'))
        model.compile(loss=self._huber_loss,
                      optimizer=Adam(lr=self.learning_rate))
        return model

    def update_target_model(self):
        # copy weights from model to target_model
        self.target_model.set_weights(self.model.get_weights())

    def remember(self, state, action, reward, next_state, done):
        self.memory.append((state, action, reward, next_state, done))

    def act(self, state):
        if np.random.rand() <= self.epsilon:
            return random.randrange(self.action_size)
        act_values = self.model.predict(state)
        return np.argmax(act_values[0])  # returns action

    def replay(self, batch_size):
        minibatch = random.sample(self.memory, batch_size)
        for state, action, reward, next_state, done in minibatch:
            target = self.model.predict(state)
            if done:
                target[0][action] = reward
            else:
                # a = self.model.predict(next_state)[0]
                t = self.target_model.predict(next_state)[0]
                target[0][action] = reward + self.gamma * np.amax(t)
                # target[0][action] = reward + self.gamma * t[np.argmax(a)]
            self.model.fit(state, target, epochs=1, verbose=0)
        if self.epsilon > self.epsilon_min:
            self.epsilon *= self.epsilon_decay

    def load(self, name):
        self.model.load_weights(name)

    def save(self, name):
        self.model.save_weights(name)


###Training

In [0]:
if __name__ == "__main__":
    env = gym.make('CartPole-v1')
    state_size = env.observation_space.shape[0]
    action_size = env.action_space.n
    agent = DQNAgent(state_size, action_size)
    done = False
    batch_size = 32

    for e in range(EPISODES):
        state = env.reset()
        state = np.reshape(state, [1, state_size])
        for time in range(500):
            # env.render()
            action = agent.act(state)
            next_state, reward, done, _ = env.step(action)
            reward = reward if not done else -10
            next_state = np.reshape(next_state, [1, state_size])
            agent.remember(state, action, reward, next_state, done)
            state = next_state
            if done:
                agent.update_target_model()
                print("episode: {}/{}, score: {}, e: {:.2}"
                      .format(e, EPISODES, time, agent.epsilon))
                break
            if len(agent.memory) > batch_size:
                agent.replay(batch_size)

episode: 0/60, score: 21, e: 1.0
episode: 1/60, score: 23, e: 0.88
episode: 2/60, score: 17, e: 0.74
episode: 3/60, score: 11, e: 0.66
episode: 4/60, score: 11, e: 0.59
episode: 5/60, score: 10, e: 0.54
episode: 6/60, score: 21, e: 0.43
episode: 7/60, score: 50, e: 0.26
episode: 8/60, score: 23, e: 0.21
episode: 9/60, score: 47, e: 0.13
episode: 10/60, score: 55, e: 0.075
episode: 11/60, score: 99, e: 0.028
episode: 12/60, score: 112, e: 0.0099
episode: 13/60, score: 183, e: 0.0099
episode: 14/60, score: 124, e: 0.0099
episode: 15/60, score: 75, e: 0.0099
episode: 16/60, score: 79, e: 0.0099
episode: 17/60, score: 98, e: 0.0099
episode: 18/60, score: 82, e: 0.0099
episode: 19/60, score: 81, e: 0.0099
episode: 20/60, score: 91, e: 0.0099
episode: 21/60, score: 122, e: 0.0099
episode: 22/60, score: 148, e: 0.0099
episode: 23/60, score: 156, e: 0.0099
episode: 24/60, score: 136, e: 0.0099
episode: 25/60, score: 150, e: 0.0099
episode: 26/60, score: 124, e: 0.0099
episode: 27/60, score: 14